In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from time import gmtime, strftime

In [48]:
df_train = pd.read_csv('train_V2.csv')
df_test = pd.read_csv('test_V2_.csv')

In [19]:
df_rand = df_train.reindex(np.random.permutation(df_train.index))

RangeIndex(start=0, stop=4446966, step=1)

In [20]:
df_rand2 = df_train.reindex()
#df_rand2.head()

In [22]:
df_rand3 = df_train.reindex(np.random.permutation(df_train.shape[0]))
df_rand3.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
2725960,33b8a88f4e92ec,9cad9c9ece46a4,ae30ae0cb80bc4,0,0,100.00,0,1,0,36,...,0,0.0,0,0.0,0,0,297.70,3,1401,0.2737
3719340,22af2bf0eaf6cc,4bb3adc78d470f,2ecbc5f1944250,0,0,100.00,0,0,0,38,...,0,0.0,0,0.0,0,0,14.84,2,1500,0.0737
390514,4fcd7263f1a5b2,6db90df573c056,4d2fa6d12a1d8d,0,0,0.00,0,0,0,72,...,0,0.0,0,0.0,0,0,622.10,4,0,0.2963
552556,370ecaab859e90,e4d8091ccb0edd,f92a421c353eb7,1,2,182.60,0,0,5,27,...,2,0.0,0,0.0,0,0,2843.00,6,0,0.9259
3953273,69e58cfeb70ef7,7f2e2e9e91e66a,04c2ca01110d5f,0,0,65.59,0,0,2,57,...,0,4718.0,0,0.0,0,0,943.80,6,1568,0.4074


### Preprocess the data for training, validation and testing

In [ ]:
X_train = df_rand3[['killPlace', 'walkDistance']].head(4000000)
Y_train = df_rand3[['winPlacePerc']].head(4000000)
X_val = df_rand3[['killPlace', 'walkDistance']].tail(446966)
Y_val = df_rand3[['winPlacePerc']].tail(446966)
X_test = df_test[['killPlace', 'walkDistance']]

### Create the feature columns

In [173]:
#help(tf.feature_column.numeric_column)

In [70]:
killPlace_feat_col = tf.feature_column.numeric_column(key='killPlace')
walkDist_feat_col = tf.feature_column.numeric_column(key='walkDistance')
feat_cols = set([killPlace_ft_col, walkDist_ft_col])

In [71]:
feat_cols

{NumericColumn(key='killPlace', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='walkDistance', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)}

### Input Functions

In [1]:
#X_train.head()

In [161]:
X_np_train = np.array(X_train)
#X_np_train[:,0]

In [3]:
help(np.array)

NameError: name 'np' is not defined

There is only 2 arguments that are neccesary in numpy_input_fn
numpy_input_fn

In [126]:
x = {'killPlace': X_np_train[:,0], 'walkDistance': X_np_train[:,1]}
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x,
    y=np.array(Y_train),
    num_epochs=None,
    shuffle=True)
#num_epochs determines what? the passes that I do along the data?
# if we set num_epochs to none it allows us to read as much data as we can from this input fn?
# shuffle randomizes the data

In [163]:
#train_input_fn()

In [164]:
X_val_np = np.array(X_val)
#X_val_np = X_val.values
#X_val_np

In [147]:
x_val_in = {'killPlace': X_val_np[:,0], 'walkDistance': X_val_np[:,1]}
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x_val_in,
    y=np.array(Y_val),
    num_epochs=1,
    shuffle=False)

In [165]:
#val_input_fn()

In [166]:
#help(tf.estimator.inputs.numpy_input_fn)

In [132]:
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x,
    shuffle=False)

In [172]:
#help(tf.estimator.DNNRegressor)

In [170]:
s = strftime("%a_%d_%b_%Y_%H:%M", gmtime())
regressor = tf.estimator.DNNRegressor(
    feature_columns=[killPlace_feat_col, walkDist_feat_col],
    hidden_units=[256],
    model_dir='./logs/'+s,
    optimizer=tf.train.ProximalAdagradOptimizer(
        learning_rate=0.1,
        l1_regularization_strength=0.001
    )
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './logs/Sun_10_Mar_2019_23:01', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f46a27652e8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [171]:
for i in range(10)
    regressor.train(
        input_fn=train_input_fn,
        steps=1000
    )
    regressor.evaluate(
        input_fn=
    )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./logs/Sun_10_Mar_2019_23:01/model.ckpt.
INFO:tensorflow:loss = 19268.635, step = 1
INFO:tensorflow:global_step/sec: 839.749
INFO:tensorflow:loss = 95.056366, step = 101 (0.119 sec)
INFO:tensorflow:global_step/sec: 1133.63
INFO:tensorflow:loss = 34.888092, step = 201 (0.088 sec)
INFO:tensorflow:global_step/sec: 982.174
INFO:tensorflow:loss = 21.026945, step = 301 (0.102 sec)
INFO:tensorflow:global_step/sec: 999.474
INFO:tensorflow:loss = 20.778072, step = 401 (0.100 sec)
INFO:tensorflow:global_step/sec: 1077.55
INFO:tensorflow:loss = 12.196705, step = 501 (0.093 sec)
INFO:tensorflow:global_step/sec: 979.033
INFO:tensorflow:loss = 16.695892, step = 601 (0.102 sec)
INFO:tensorflow:global_step/sec: 934.499
INFO

In [64]:
help(tf.estimator.DNNRegressor.train)

Help on function train in module tensorflow_estimator.python.estimator.estimator:

train(self, input_fn, hooks=None, steps=None, max_steps=None, saving_listeners=None)
    Trains a model given training data `input_fn`.
    
    Args:
      input_fn: A function that provides input data for training as minibatches.
        See [Premade Estimators](
        https://tensorflow.org/guide/premade_estimators#create_input_functions)
        for more information. The function should construct and return one of
        the following:  * A
        `tf.data.Dataset` object: Outputs of `Dataset` object must be a tuple
        `(features, labels)` with same constraints as below. * A tuple
        `(features, labels)`: Where `features` is a `tf.Tensor` or a dictionary
        of string feature name to `Tensor` and `labels` is a `Tensor` or a
        dictionary of string label name to `Tensor`. Both `features` and
        `labels` are consumed by `model_fn`. They should satisfy the expectation
       